In [1]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-98-93-17-21.compute-1.amazonaws.com:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("05 - Model Selection & Basic Hyperparameter Tuning")

<Experiment: artifact_location='s3://sau-s3-mlflow-bucket/309540500998410326', creation_time=1763854853571, experiment_id='309540500998410326', last_update_time=1763854853571, lifecycle_stage='active', name='05 - Model Selection & Basic Hyperparameter Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [3]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv(
    '../../data/processed/reddit_clean_final.csv',
    keep_default_na=False,
    na_filter=False
)
df.shape

(36607, 2)

In [ ]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 2)  # Bigram setting
max_features = 10000  # Set max_features to 10000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=5)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(tree_method="hist" ,n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
[I 2025-11-23 18:03:25,545] A new study created in memory with name: no-name-09c8b977-dce6-47e0-9afc-849faee235ed
[I 2025-11-23 18:08:37,495] Trial 0 finished with value: 0.7055449330783938 and parameters: {'n_estimators': 263, 'learning_rate': 0.04124206172922066, 'max_depth': 3}. Best is trial 0 with value: 0.7055449330783938.
[I 2025-11-23 18:18:54,341] Trial 1 finished with value: 0.7323135755258127 and parameters: {'n_estimators': 140, 'learning_rate': 0.061141029880433406, 'max_depth': 6}. Best is trial 1 with value: 0.7323135755258127.
[I 2025-11-23 18:22:56,417] Trial 2 finished with value: 0.5971046162250752 and parameters: {'n_esti

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-98-93-17-21.compute-1.amazonaws.com:5000/#/experiments/309540500998410326/runs/1d30b6f2fcf542d682d988c9be7ac421
🧪 View experiment at: http://ec2-98-93-17-21.compute-1.amazonaws.com:5000/#/experiments/309540500998410326
